In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [6]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [8]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list), 1)
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(12240, 37818)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [9]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [10]:
test_genres = one_hot_genres(test_df['Genre'])

In [11]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size, 1)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 37818)


## **Don't rerun the cell below, as this will result in adding dimensions to the sets, which will not work! Can rerun all other cells except this one.**

In [15]:
cnn_lyrics_train = train_lyrics[:, :, None]
cnn_lyrics_test = test_lyrics[:, :, None]

## Convolutional Neural Network

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(Conv1D(128, kernel_size=(1), activation='relu', input_shape=(len(word_list),1)))
model.add(Conv1D(32, (1), activation='relu'))
model.add(Dropout(0.8)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(16, activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 37818, 128)        256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 37818, 32)         4128      
_________________________________________________________________
dropout_5 (Dropout)          (None, 37818, 32)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1210176)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                19362832  
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                

In [29]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(cnn_lyrics_train, train_genres, epochs=5, batch_size=16, validation_split=17/100)

Train on 10159 samples, validate on 2081 samples
Epoch 1/5
10159/10159 [==============================] - 365s 36ms/sample - loss: 1.8200 - acc: 0.3157 - val_loss: 1.5623 - val_acc: 0.4330
Epoch 2/5
10159/10159 [==============================] - 363s 36ms/sample - loss: 1.4720 - acc: 0.4619 - val_loss: 1.3705 - val_acc: 0.5026
Epoch 3/5
10159/10159 [==============================] - 352s 35ms/sample - loss: 1.2486 - acc: 0.5457 - val_loss: 1.3677 - val_acc: 0.5060
Epoch 4/5
10159/10159 [==============================] - 361s 36ms/sample - loss: 1.1196 - acc: 0.5907 - val_loss: 1.3573 - val_acc: 0.5050
Epoch 5/5
10159/10159 [==============================] - 375s 37ms/sample - loss: 1.0298 - acc: 0.6244 - val_loss: 1.3960 - val_acc: 0.5099


In [30]:
results = model.evaluate(cnn_lyrics_test, test_genres, batch_size=16)

2160/2160 [==============================] - 19s 9ms/sample - loss: 1.4011 - acc: 0.5088


In [31]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(cnn_lyrics_test)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

KeyboardInterrupt: 

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values